# Example of merging dfs

In [42]:
import pandas as pd
import numpy as np
import sqlite3

In [43]:
# Sample df

patient_df = pd.DataFrame({
    'PatientID' : range(1, 51),
    'BloodPressure': np.random.randint(80, 180, 50),
    'Smoking': np.random.randint(0, 2, 50)
    # 'WhiteBloodCellCount': np.random.randint(300, 800, 50),
    # 'CholesterolLevels': np.random.randint(5, 30, 50)
})

BloodPressure_range_df = pd.DataFrame({
    'BloodPressureDiagnosisID': range(3),
    'Low': [0, 121, 141],
    'High': [120, 140, 5000]
})

BloodPressureDiagnosis_df = pd.DataFrame({
    'BloodPressureDiagnosisID': range(3),
    'BloodPressureDiagnosis': ['No treatment needed', 'Exercise Daily', 'You\'re screwed!']
})

SmokingDiagnosis_df = pd.DataFrame({
    'Smoking': [0, 1],
    'SmokingDiagnosis': ['Good job!', 'STOP IT NOW DUMMY!']
})


In [44]:
patient_df

,PatientID,BloodPressure,Smoking
0,1,104,0
1,2,110,1
2,3,172,0
3,4,148,0
4,5,98,0
5,6,84,1
6,7,114,0
7,8,156,1
8,9,160,0
9,10,119,1


In [45]:
BloodPressure_range_df

,BloodPressureDiagnosisID,Low,High
0,0,0,120
1,1,121,140
2,2,141,5000


In [46]:
BloodPressureDiagnosis_df

,BloodPressureDiagnosisID,BloodPressureDiagnosis
0,0,No treatment needed
1,1,Exercise Daily
2,2,You're screwed!


In [47]:
SmokingDiagnosis_df

,Smoking,SmokingDiagnosis
0,0,Good job!
1,1,STOP IT NOW DUMMY!


In [48]:
# There's no good way in pandas to join on a range of values, so easiest to use sql for any part involving a range. The rest can be done easily with merges.

conn = sqlite3.connect(':memory:')
patient_df.to_sql("patient_df", conn, index=False)
BloodPressure_range_df.to_sql("BloodPressure_range_df", conn, index=False)
query = "SELECT * FROM patient_df LEFT JOIN BloodPressure_range_df ON patient_df.BloodPressure BETWEEN BloodPressure_range_df.Low AND BloodPressure_range_df.High"
new_patient_df = pd.read_sql_query(query, conn)

new_patient_df

,PatientID,BloodPressure,Smoking,BloodPressureDiagnosisID,Low,High
0,1,104,0,0,0,120
1,2,110,1,0,0,120
2,3,172,0,2,141,5000
3,4,148,0,2,141,5000
4,5,98,0,0,0,120
5,6,84,1,0,0,120
6,7,114,0,0,0,120
7,8,156,1,2,141,5000
8,9,160,0,2,141,5000
9,10,119,1,0,0,120


In [50]:
new_patient_df.merge(BloodPressureDiagnosis_df).merge(SmokingDiagnosis_df).sort_values('PatientID')

,PatientID,BloodPressure,Smoking,BloodPressureDiagnosisID,Low,High,BloodPressureDiagnosis,SmokingDiagnosis
0,1,104,0,0,0,120,No treatment needed,Good job!
27,2,110,1,0,0,120,No treatment needed,STOP IT NOW DUMMY!
12,3,172,0,2,141,5000,You're screwed!,Good job!
13,4,148,0,2,141,5000,You're screwed!,Good job!
1,5,98,0,0,0,120,No treatment needed,Good job!
28,6,84,1,0,0,120,No treatment needed,STOP IT NOW DUMMY!
2,7,114,0,0,0,120,No treatment needed,Good job!
40,8,156,1,2,141,5000,You're screwed!,STOP IT NOW DUMMY!
14,9,160,0,2,141,5000,You're screwed!,Good job!
29,10,119,1,0,0,120,No treatment needed,STOP IT NOW DUMMY!
